In [2]:
%load_ext autoreload
%autoreload 2

### Imports

In [3]:
%matplotlib inline

import IPython.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import torch
import torch.optim
from torch.utils.data import DataLoader

from utils import log_textfile,load_model
from basetrainer import BaseTrainer
from dataloader import load_data
from CustomDataset import CustomDatasetSimple
from models.resnet import resnet34
from attacks import fgsm, pgd_linf

Load data and prepare train and validation set

In [18]:
DATADIR = '../../Data/'
data = load_data(DATADIR)

In [9]:
trainset = pd.read_csv('filesnames/trainset_10_uids.csv')
valset = pd.read_csv('filesnames/valset_10_uids.csv')

trainset = trainset.drop('Unnamed: 0', axis=1)
valset = valset.drop('Unnamed: 0', axis=1)

In [12]:
trainset_dl = [tuple(x) for x in trainset.values]
valset_dl = [tuple(x) for x in valset.values]

train_filepaths = [i[2] for i in trainset_dl]
train_labels = [i[3] for i in trainset_dl]
valid_filepaths = [i[2] for i in valset_dl]
val_labels = [i[3] for i in valset_dl]

In [13]:
# from utils import get_mean_std
#mean, std = get_mean_std(train_data_loader)

mean=-3.1259581955996425
std=0.8961027914827521
batch_size=16
num_workers=8
data_train_sub = CustomDatasetSimple(train_filepaths, train_labels, mean, std)
data_valid_sub = CustomDatasetSimple(valid_filepaths, val_labels, mean, std)

train_data_loader = torch.utils.data.DataLoader(
    dataset=data_train_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

valid_data_loader = torch.utils.data.DataLoader(
    dataset=data_valid_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

In [15]:
MODELNAME = '10_uids_normal_10_epochs_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

In [130]:
trainer.fit_model_new(optimizer=torch.optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=10, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=None, 
                      epsilon=None, 
                      alpha=None, 
                      num_iter=None)

20
[epoch: 1, batch:     20] loss: 0.11023 time model: 0.02542 acc: 0.12031
40
[epoch: 1, batch:     40] loss: 0.10049 time model: 0.02540 acc: 0.16250
60
[epoch: 1, batch:     60] loss: 0.09386 time model: 0.02540 acc: 0.18594
80
[epoch: 1, batch:     80] loss: 0.09003 time model: 0.02539 acc: 0.20898
100
[epoch: 1, batch:    100] loss: 0.08579 time model: 0.02537 acc: 0.23906
epoch:1 train loss: 0.08470685479970404 train acc: 0.2447572536627406 valid loss: 0.1669788106282552 valid acc: 0.16
epoch:1 train loss: 0.08470685479970404 train acc: 0.2447572536627406 valid loss: 0.1669788106282552 valid acc: 0.16
20
[epoch: 2, batch:     20] loss: 0.06293 time model: 0.02542 acc: 0.37969
40
[epoch: 2, batch:     40] loss: 0.06018 time model: 0.02540 acc: 0.42266
60
[epoch: 2, batch:     60] loss: 0.05780 time model: 0.02540 acc: 0.45104
80
[epoch: 2, batch:     80] loss: 0.05560 time model: 0.02539 acc: 0.46172
100
[epoch: 2, batch:    100] loss: 0.05310 time model: 0.02539 acc: 0.48812
epoc

In [134]:
MODELNAME = '10_uids_fgsm_.05_10_epochs_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

In [135]:
trainer.fit_model_new(optimizer=torch.optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=10, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=fgsm, 
                      epsilon=0.05, 
                      alpha=None, 
                      num_iter=None)

20
[epoch: 1, batch:     20] loss: 0.12101 time model: 0.05034 acc: 0.06875
40
[epoch: 1, batch:     40] loss: 0.11054 time model: 0.05035 acc: 0.10391
60
[epoch: 1, batch:     60] loss: 0.10574 time model: 0.05037 acc: 0.11302
80
[epoch: 1, batch:     80] loss: 0.10148 time model: 0.05037 acc: 0.13437
100
[epoch: 1, batch:    100] loss: 0.09825 time model: 0.05036 acc: 0.14719
epoch:1 train loss: 0.09697509120162004 train acc: 0.15800057454754382 valid loss: 0.17395661354064942 valid acc: 0.16
epoch:1 train loss: 0.09697509120162004 train acc: 0.15800057454754382 valid loss: 0.17395661354064942 valid acc: 0.16
20
[epoch: 2, batch:     20] loss: 0.07569 time model: 0.05036 acc: 0.28906
40
[epoch: 2, batch:     40] loss: 0.07302 time model: 0.05035 acc: 0.30703
60
[epoch: 2, batch:     60] loss: 0.07171 time model: 0.05035 acc: 0.31250
80
[epoch: 2, batch:     80] loss: 0.06912 time model: 0.05034 acc: 0.32461
100
[epoch: 2, batch:    100] loss: 0.06677 time model: 0.05035 acc: 0.34094


In [136]:
MODELNAME = '10_uids_10_epochs_fgsm_0.1_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

In [137]:
trainer.fit_model_new(optimizer=torch.optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=10, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=fgsm, 
                      epsilon=0.1, 
                      alpha=None, 
                      num_iter=None)

20
[epoch: 1, batch:     20] loss: 0.11768 time model: 0.05023 acc: 0.07344
40
[epoch: 1, batch:     40] loss: 0.10713 time model: 0.05022 acc: 0.11094
60
[epoch: 1, batch:     60] loss: 0.10085 time model: 0.05021 acc: 0.13802
80
[epoch: 1, batch:     80] loss: 0.09690 time model: 0.05021 acc: 0.15781
100
[epoch: 1, batch:    100] loss: 0.09309 time model: 0.05022 acc: 0.18125
epoch:1 train loss: 0.09172494560094854 train acc: 0.1910370583165757 valid loss: 0.10466446876525878 valid acc: 0.19333333333333333
epoch:1 train loss: 0.09172494560094854 train acc: 0.1910370583165757 valid loss: 0.10466446876525878 valid acc: 0.19333333333333333
20
[epoch: 2, batch:     20] loss: 0.07140 time model: 0.05040 acc: 0.33281
40
[epoch: 2, batch:     40] loss: 0.06838 time model: 0.05039 acc: 0.33672
100
[epoch: 2, batch:    100] loss: 0.06023 time model: 0.05042 acc: 0.41219
epoch:2 train loss: 0.059670716104614295 train acc: 0.4179833381212295 valid loss: 0.10539337952931722 valid acc: 0.24666666

In [138]:
MODELNAME = '10_uids_10_epochs_fgsm_0.3_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

In [139]:
trainer.fit_model_new(optimizer=torch.optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=10, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=fgsm, 
                      epsilon=0.3, 
                      alpha=None, 
                      num_iter=None)

20
[epoch: 1, batch:     20] loss: 0.12218 time model: 0.27509 acc: 0.04844
40
[epoch: 1, batch:     40] loss: 0.11230 time model: 0.27509 acc: 0.07266
60
[epoch: 1, batch:     60] loss: 0.10753 time model: 0.27521 acc: 0.08854
80
[epoch: 1, batch:     80] loss: 0.10415 time model: 0.27524 acc: 0.10938
100
[epoch: 1, batch:    100] loss: 0.10113 time model: 0.27529 acc: 0.12406
epoch:1 train loss: 0.1001495140374854 train acc: 0.1315713875323183 valid loss: 0.10711635907491047 valid acc: 0.12
epoch:1 train loss: 0.1001495140374854 train acc: 0.1315713875323183 valid loss: 0.10711635907491047 valid acc: 0.12
20
[epoch: 2, batch:     20] loss: 0.08300 time model: 0.27537 acc: 0.22656
40
[epoch: 2, batch:     40] loss: 0.07949 time model: 0.27558 acc: 0.25391
60
[epoch: 2, batch:     60] loss: 0.07792 time model: 0.27567 acc: 0.26875
80
[epoch: 2, batch:     80] loss: 0.07658 time model: 0.27571 acc: 0.27734
100
[epoch: 2, batch:    100] loss: 0.07444 time model: 0.27571 acc: 0.29219
epoc

In [28]:
MODELNAME = '10_uids_10_epochs_pgd_0.05_0.02_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

In [29]:
trainer.fit_model_new(optimizer=torch.optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=10, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=pgd_linf, 
                      epsilon=0.05, 
                      alpha=0.02, 
                      num_iter=10)

20
[epoch: 1, batch:     20] loss: 0.12136 time model: 0.23699 acc: 0.05312
40
[epoch: 1, batch:     40] loss: 0.11239 time model: 0.23313 acc: 0.07891
60
[epoch: 1, batch:     60] loss: 0.10858 time model: 0.23223 acc: 0.08906
80
[epoch: 1, batch:     80] loss: 0.10608 time model: 0.23187 acc: 0.09414
100
[epoch: 1, batch:    100] loss: 0.10369 time model: 0.23161 acc: 0.10375
epoch:1 train loss: 0.10287262447535803 train acc: 0.10830221200804367 valid loss: 0.09774765014648437 valid acc: 0.18
epoch:1 train loss: 0.10287262447535803 train acc: 0.10830221200804367 valid loss: 0.09774765014648437 valid acc: 0.18
20
[epoch: 2, batch:     20] loss: 0.08369 time model: 0.23015 acc: 0.18906
40
[epoch: 2, batch:     40] loss: 0.07998 time model: 0.23028 acc: 0.23047
60
[epoch: 2, batch:     60] loss: 0.07713 time model: 0.23009 acc: 0.25052
80
[epoch: 2, batch:     80] loss: 0.07448 time model: 0.23007 acc: 0.27813
100
[epoch: 2, batch:    100] loss: 0.07293 time model: 0.23009 acc: 0.28969


KeyboardInterrupt: 

In [ ]:
MODELNAME = '10_uids_10_epochs_pgd_0.1_0.05_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

In [ ]:
trainer.fit_model_new(optimizer=torch.optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=10, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=pgd_linf, 
                      epsilon=0.1, 
                      alpha=0.05, 
                      num_iter=10)

In [ ]:
MODELNAME = '10_uids_10_epochs_pgd_0.3_0.1_sp'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

### Evaluation

In [40]:
models = [
    '10_uids_normal_10_epochs_sp',
    '10_uids_fgsm_.05_10_epochs_sp',
    '10_uids_10_epochs_fgsm_0.1_sp',
    '10_uids_10_epochs_fgsm_0.3_sp',
    '10_uids_10_epochs_pgd_0.05_0.02_sp',
    '10_uids_10_epochs_pgd_0.1_0.05_sp',
    '10_uids_10_epochs_pgd_0.3_0.1_sp'
]

In [41]:
attack_params = {
    'none': (None, None, None, None),
    'fgsm_.05': (fgsm, 0.05, None, None),
    'fgsm_.1': (fgsm, 0.1, None, None),
    'fgsm_.3': (fgsm, 0.3, None, None),
    'pgd_.05_.02': (pgd_linf, 0.05, 0.02, 20),
    'pgd_.1_.05': (pgd_linf, 0.1, 0.05, 20),
    'pgd_.3_.1': (pgd_linf, 0.3, 0.1, 20)
}

In [42]:
results = {}

for model_name in models:
    print(model_name)
    model = load_model('saved/' + str(model_name))
    model = model.eval()
    LOGFILE_PATH = MODELNAME + '_eval'
    trainer = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)
    model_results = {}
    for param_key in attack_params.keys():
        params = attack_params[param_key]
        attack = params[0]
        epsilon = params[1]
        alpha = params[2]
        num_iter = params[3]
        
        loss, acc = trainer.run_epoch(0, valid_data_loader, LOGFILE_PATH, optimizer=None, attack=attack, 
                          epsilon=epsilon, alpha=alpha, num_iter=num_iter)
        
        if attack!=fgsm:
            model_results['pgd' + '_eps_' + str(epsilon) + '_alpha_' + str(alpha) + '_num_iter_' + str(num_iter)] = acc
        else:
            model_results['fgsm' + '_eps_' + str(epsilon)] = acc

            
    results[model_name] = model_results
    
results_df = pd.DataFrame(results).T
results_df.to_csv('results_speaker_verification_spectrograms.csv')
       

30_uids_normal_10_epochs_lr_0.001
30_uids_fgsm_05_10_epochs_lr_0.001
30_uids_10_epochs_lr_0.001_fgsm_01
30_uids_epochs_lr_0.001_pgd_05_05
